<a href="https://colab.research.google.com/github/ViktoryLoktionova/Keras_tuner_fashion_mnist/blob/main/Keras_tuner_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/keras-team/keras-tuner/

Cloning into 'keras-tuner'...
remote: Enumerating objects: 7415, done.
remote: Counting objects: 100% (773/773), done.
remote: Compressing objects: 100% (387/387), done.
remote: Total 7415 (delta 480), reused 587 (delta 359), pack-reused 6642
Receiving objects: 100% (7415/7415), 1.63 MiB | 1.15 MiB/s, done.
Resolving deltas: 100% (5185/5185), done.


Установка Keras Tuner

In [ ]:
pip install -U keras-tuner

     |████████████████████████████████| 97 kB 3.2 MB/s 


In [ ]:
%tensorflow_version 2.x
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import utils
from google.colab import files
from kerastuner.tuners import RandomSearch, Hyperband, BayesianOptimization
import numpy as np

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import sys


Подготовка данных для обучения сети

In [ ]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

4431872/4422102 [==============================] - 0s 0us/step


In [ ]:
x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train / 255 
x_test = x_test / 255 
y_train = utils.to_categorical(y_train, 10)
y_test = utils.to_categorical(y_test, 10)

Задаем функцию создания нейронной сети


In [ ]:
def build_model(hp):
    model = Sequential()
    activation_choice = hp.Choice('activation', values=['relu', 'sigmoid', 'tanh', 'elu', 'selu'])    
    model.add(Dense(units=hp.Int('units_input',    # Полносвязный слой с разным количеством нейронов
                                   min_value=512,    # минимальное количество нейронов - 512
                                   max_value=1024,   # максимальное количество - 1024
                                   step=32),
                    input_dim=784,
                    activation=activation_choice))
    model.add(Dense(units=hp.Int('units_hidden',        
                                   min_value=128,   
                                   max_value=600,   
                                   step=32),
                    activation=activation_choice))   
    model.add(Dense(10, activation='softmax'))
    model.compile(
        optimizer=hp.Choice('optimizer', values=['adam','rmsprop','SGD']),
        loss='categorical_crossentropy',
        metrics=['accuracy'])
    return model

Создаем tuner

In [ ]:
tuner = RandomSearch(
    build_model,                 # функция создания модели
    objective='val_accuracy',    # метрика, которую нужно оптимизировать - 
                                 # доля правильных ответов на проверочном наборе данных
    max_trials=3,               # максимальное количество запусков обучения 
    directory='test_directory'   # каталог, куда сохраняются обученные сети  
    )

INFO:tensorflow:Reloading Oracle from existing project test_directory/untitled_project/oracle.json
INFO:tensorflow:Reloading Tuner from test_directory/untitled_project/tuner0.json


Запускаем подбор гиперпараметров

In [ ]:
tuner.search_space_summary()

Search space summary
Default search space size: 4
activation (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'sigmoid', 'tanh', 'elu', 'selu'], 'ordered': False}
units_input (Int)
{'default': None, 'conditions': [], 'min_value': 512, 'max_value': 1024, 'step': 32, 'sampling': None}
units_hidden (Int)
{'default': None, 'conditions': [], 'min_value': 128, 'max_value': 600, 'step': 32, 'sampling': None}
optimizer (Choice)
{'default': 'adam', 'conditions': [], 'values': ['adam', 'rmsprop', 'SGD'], 'ordered': False}


In [ ]:
tuner.search(x_train,                  # Данные для обучения
             y_train,                  # Правильные ответы
             batch_size=256,           # Размер мини-выборки
             epochs=20,                # Количество эпох обучения 
             validation_split=0.2,     # Часть данных, которая будет использоваться для проверки
             )

Trial 1 Complete [00h 00m 41s]
val_accuracy: 0.8480833172798157

Best val_accuracy So Far: 0.9003333449363708
Total elapsed time: 00h 00m 41s
INFO:tensorflow:Oracle triggered exit


In [ ]:
tuner.results_summary()

Results summary
Results in test_directory/untitled_project
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
activation: relu
units_input: 736
units_hidden: 352
optimizer: rmsprop
Score: 0.9003333449363708
Trial summary
Hyperparameters:
activation: tanh
units_input: 512
units_hidden: 544
optimizer: adam
Score: 0.8989999890327454
Trial summary
Hyperparameters:
activation: tanh
units_input: 576
units_hidden: 352
optimizer: adam
Score: 0.8982499837875366
Trial summary
Hyperparameters:
activation: relu
units_input: 768
units_hidden: 224
optimizer: adam
Score: 0.8979166746139526
Trial summary
Hyperparameters:
activation: relu
units_input: 896
units_hidden: 544
optimizer: adam
Score: 0.8973333239555359
Trial summary
Hyperparameters:
activation: relu
units_input: 640
units_hidden: 352
optimizer: adam
Score: 0.8969166874885559
Trial summary
Hyperparameters:
activation: tanh
units_input: 608
units_hidden: 544
optimizer: adam
Score: 0.896833360

Получаем три лучших модели

In [ ]:
models = tuner.get_best_models(num_models=3)

Оцениваем качество модели на тестовых данных


In [ ]:
for model in models:
  model.summary()
  model.evaluate(x_test, y_test)
  print()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 736)               577760    
_________________________________________________________________
dense_1 (Dense)              (None, 352)               259424    
_________________________________________________________________
dense_2 (Dense)              (None, 10)                3530      
Total params: 840,714
Trainable params: 840,714
Non-trainable params: 0
_________________________________________________________________
313/313 [==============================] - 1s 3ms/step - loss: 0.3691 - accuracy: 0.8908

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dense_